In [ ]:
import torch
from torch import nn
from transformers import LlamaTokenizerFast, PaliGemmaProcessor
from transformers.models.paligemma.configuration_paligemma import PaliGemmaConfig
from transformers.models.paligemma.modeling_paligemma import PaliGemmaForConditionalGeneration, PaliGemmaPreTrainedModel
import os
from PIL import Image
import requests

## Matcha Model

In [3]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/matcha-chartqa")  # Không cần .to(0)
processor = AutoProcessor.from_pretrained("google/matcha-chartqa")

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/20294671002019.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Is the sum of all 4 places greater than Belginum?",
                   return_tensors="pt")  # Không cần .to(0)

predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))


KeyboardInterrupt: 

In [34]:
text = "What is the death rate of Laos?" # Correct
text = "Is the sum of all 4 places greater than Belginum? Explain then answer" # Wrong
text = "What is the death rate of Australia?" #  True
text = "Is the death rate of Laos greater than Comoros?" #  True
text = "What is the sum of death rate of Australia and Comoros? Explain your answer" #  Wrong
text = "Noem op the death rate of Laos and Nicaragua" # Correct

In [35]:
inputs = processor(images=image, text=text,
                   return_tensors="pt")  # Không cần .to(0)

predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

68


## PaliGemma

### Single-image Inference

In [1]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

model_id = "google/paligemma-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

prompt = "What is on the flower?"
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg?download=true"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image, prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=20)

print(processor.decode(output[0], skip_special_tokens=True)[inputs.input_ids.shape[1]: ])

KeyboardInterrupt: 

### Multi-image Inference

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, PaliGemmaProcessor

# Load the model and processor
model_id = "google/paligemma-3b-ft-nlvr2-448"  # checkpoint tuned for multiple images
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
processor = PaliGemmaProcessor.from_pretrained(model_id)

# Prompt
prompt = "answer en Which of the two pictures shows a snowman, first or second?"

# Load the images from URLs
stop_sign_image = Image.open(requests.get("https://www.ilankelman.org/stopsigns/australia.jpg", stream=True).raw)
snow_image = Image.open(requests.get("https://haycafe.vn/wp-content/uploads/2021/12/Hinh-anh-nguoi-tuyet-Giang-Sinh-ma-hong-cute-800x500.jpg", stream=True).raw)

# Resize images to make them smaller (e.g., 224x224)
new_size = (224, 224)
stop_sign_image = stop_sign_image.resize(new_size)
snow_image = snow_image.resize(new_size)

# Preprocess the images and prompt text
inputs = processor(images=[[snow_image, stop_sign_image]], text=prompt, return_tensors="pt")

# Generate output from the model
output = model.generate(**inputs, max_new_tokens=20)

# Decode and print the result
result = processor.decode(output[0], skip_special_tokens=True)
# Remove the prompt from the generated result (optional)
answer = result[inputs.input_ids.shape[1]:]
print(answer)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## google/deplot

In [ ]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

In [22]:
import torch
print(torch.__version__)

2.4.1+cu118
